In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import scipy.stats
import re

## Převod exportovaných receptů na unikátní kombinaci lék-pacient 

Soubor 'rcp_export.txt' je export receptů za posledních 15 let.

In [2]:
recepty = []
with open('rcp_export.txt', encoding='Windows-1250') as f:
    for line in f:
        parts = line.split("\",\"")
        
        if len(parts)>= 9:
            rc, lek1, lek2 = parts[3], parts[6], parts[9]
            rc = rc.replace("\n", "")
            lek1 = lek1.replace("\n", "").replace(",", ".")
            lek2 = lek2.replace("\n", "").replace(",", ".")
            if lek1:
                recepty.append( (rc, lek1) )
            if lek2:
                recepty.append( (rc, lek2) )

## Čištění dat

In [3]:
with open('recepty.csv', 'w') as o:
    o.write("rc,lek\n")
    for rc, lek in recepty:
        o.write(f"{rc},{lek}\n")

In [4]:
df = pd.read_csv('recepty.csv')
df.dropna(inplace=True)
df['rc'] = df['rc'].apply(str)

## Převod rodného čísla na pohlaví

In [5]:
def getGender( rc ):
    if int(rc[2:4]) > 50:
        return 'female'
    else:
        return 'male'

In [6]:
df['gender'] = df['rc'].apply(getGender)

## Základní popisná statistika celého souboru 

In [7]:
pocet_receptu = len(df)
print(f"Počet receptů: {pocet_receptu}")

df.drop_duplicates(inplace=True)
pocet_kombinaci = len(df)
print(f"Počet kombinací pacient-lék: {pocet_kombinaci}")

pacienti_unique = pd.DataFrame( {'rc': df['rc'].unique() } )
pacienti_unique['gender'] = pacienti_unique['rc'].apply(getGender)
pocet_pacientu = len(pacienti_unique)
print(f"Počet unikátních pacientů: {pocet_pacientu}")

recepty_zeny = len(pacienti_unique.loc[pacienti_unique['gender'] == 'female'])
procent_zen_soubor = 100.0 * (recepty_zeny/pocet_pacientu)
recepty_muzi = len(pacienti_unique.loc[pacienti_unique['gender'] == 'male'])
procent_muzu_soubor = 100.0 * (recepty_muzi/pocet_pacientu)
print(f"Počet unikátních žen, kterým se předepisovaly recepty: {recepty_zeny} ({procent_zen_soubor:.1f} %).")
print(f"Počet unikátních mužů, kterým se předepisovaly recepty: {recepty_muzi} ({procent_muzu_soubor:.1f} %)")

with open('leky.txt', 'w') as o:
    for lek in df['lek'].unique():
        o.write(f"{lek}\n")

Počet receptů: 30170
Počet kombinací pacient-lék: 21411
Počet unikátních pacientů: 8568
Počet unikátních žen, kterým se předepisovaly recepty: 4948 (57.7 %).
Počet unikátních mužů, kterým se předepisovaly recepty: 3620 (42.3 %)


## Výber těch léků, ktere jsou na bolest 

In [8]:
def getMedication( lek ):
    """
        Vypíše účinnou látku léku podle údajů SÚKL
    """
    url = 'http://www.sukl.cz/modules/medication/search.php?data%5Bchbox%5D%5B%5D=marketability&data%5Bsearch_for%5D='
    url = 'http://www.sukl.cz/modules/medication/search.php?data%5Bsearch_for%5D='
    try:
        lek = lek.split(" ")[0]
        r = requests.get(url = url+lek)
        soup = BeautifulSoup(r.text, 'html.parser')
        td = soup.find('td', {'headers': 'nazev'})
        a = td.find('a')
        href = a['href']
        time.sleep(2)

        r = requests.get(url = href)
        soup = BeautifulSoup(r.text, 'html.parser')
        text = soup.find('th', text = u'Účinná látka').fetchNextSiblings()[0].get_text()
        time.sleep(2)
        return text.replace("<td>", "").replace("</td>", "").replace(" ", "").strip()
    except Exception:
        print(f"*{lek}")

In [ ]:
with open('leky.txt') as f:
    for line in f:
        lek = line.strip()
        med = getMedication(lek)
        print(f"{lek}, {med}")

Soubor "leky.txt" upraven tak, aby obsahoval jen léky na bolest jako "leky_na_bolest.txt".

In [9]:
leky_na_bolest = []
with open('leky_na_bolest.txt') as f:
    for line in f: leky_na_bolest.append(line.strip())
pocet_leku_na_bolest = len(leky_na_bolest)
print(f"Počet unikátních léků: {pocet_leku_na_bolest}")

Počet unikátních léků: 215


Vyber pouze ty recepty, které jsou na lék na bolest

In [10]:
len(df[ df['lek'].isin(leky_na_bolest)])

8183

In [11]:
len(df[ ~df['lek'].isin(leky_na_bolest)])

13228

In [12]:
df = df[ df['lek'].isin(leky_na_bolest) ]

In [13]:
len(df)

8183

## Pohlaví u léků na bolest

In [14]:
df['gender'] = df['rc'].apply(getGender)

In [15]:
bolest_muzi = len(df.loc[ df['gender'] == 'male', 'rc'].unique())
bolest_zeny = len(df.loc[ df['gender'] == 'female', 'rc'].unique())
bolest_procent_muzi = 100.0 * (bolest_muzi/len(df['rc'].unique()))
bolest_procent_zeny = 100.0 * (bolest_zeny/len(df['rc'].unique()))
bolest_celkem = len(df['rc'].unique())
print(f"Počet lidí s léky na bolest: {bolest_celkem}")
print(f"Počet žen s léky na bolest: {bolest_zeny} ({bolest_procent_zeny:.1f} %)")
print(f"Počet mužů s léky na bolest: {bolest_muzi} ({bolest_procent_muzi:.1f} %)")

Počet lidí s léky na bolest: 5279
Počet žen s léky na bolest: 3151 (59.7 %)
Počet mužů s léky na bolest: 2128 (40.3 %)


In [16]:
kontingencni_tabulka = [
        [bolest_muzi, bolest_zeny],
        [recepty_muzi, recepty_zeny]
    ]
kontingencni_tabulka

[[2128, 3151], [3620, 4948]]

In [17]:
oddsratio, pvalue = scipy.stats.fisher_exact(
    [
        [bolest_muzi, bolest_zeny],
        [recepty_muzi, recepty_zeny]
    ]
)
print(f"P-hodnota: {pvalue}")

P-hodnota: 0.02527689026555753


## Návštěvy pacientů 

In [18]:
isRc = re.compile('\d{9,10}')

In [19]:
rodna_cisla = []
with open('amb_navs_2004-09-15_2019-09-15.txt', encoding='Windows-1250') as f:
    for line in f:
        if "|" in line:
            jmeno, rc, *_ = line.split("|")
            if isRc.match(rc): rodna_cisla.append(rc)

In [20]:
rodna_cisla = set(rodna_cisla)
pohlavi = map(lambda p: 1 if getGender(p) == 'female' else 0, rodna_cisla)
pohlavi = list(pohlavi)

In [21]:
zeny = sum(pohlavi)
zeny_procent = 100 * (zeny / len(pohlavi) )

In [22]:
print(f"Zdravotnické zařízení navštívilo {zeny_procent:.1f} % žen.")

Zdravotnické zařízení navštívilo 55.9 % žen.
